In [1]:
for i in range(1,101):
    if i%3 == 0 and i%5 == 0:
        print("FizzBuzz")
    elif i%5 == 0:
        print("Buzz")
    elif i%3 == 0:
        print("Fizz")
    else:
        print(i)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz
Fizz
52
53
Fizz
Buzz
56
Fizz
58
59
FizzBuzz
61
62
Fizz
64
Buzz
Fizz
67
68
Fizz
Buzz
71
Fizz
73
74
FizzBuzz
76
77
Fizz
79
Buzz
Fizz
82
83
Fizz
Buzz
86
Fizz
88
89
FizzBuzz
91
92
Fizz
94
Buzz
Fizz
97
98
Fizz
Buzz


In [2]:
import execnet


def square(channel):
    while not channel.isclosed():
        number = channel.recieve()
        number_squared = number**2
        channel.send(number_squared)

In [3]:
gateway = execnet.makegateway()
channel = gateway.remote_exec(square)

In [4]:
for i in range(10):
    channel.send(i)
    i_squared = channel.recieve()
    print(f"{i} squared is {i_squared}")

OSError: cannot send to <Channel id=1 closed>

In [5]:
gateway.exit()

In [6]:
import multiprocessing


def square_mp(in_queue, out_queue):
    while True:
        n = in_queue.get()
        n_squared = n**2
        out_queue.put(n_squared)
        

if __name__ == '__main__':
    in_queue = multiprocessing.Queue()
    out_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=square_mp, args=(in_queue, out_queue))
    process.start()
    for i in range(10):
        in_queue.put(i)
        i_squared = out_queue.get()
        print(f"{i} squared is {i_squared}")
    process.terminate()

0 squared is 0
1 squared is 1
2 squared is 4
3 squared is 9
4 squared is 16
5 squared is 25
6 squared is 36
7 squared is 49
8 squared is 64
9 squared is 81


In [7]:
import threading
import queue

in_queue = queue.Queue()
out_queue = queue.Queue()

def square_threading():
    while True:
        n = in_queue.get()
        if n == 'STOP':
            return
        n_squared = n**2
        out_queue.put(n_squared)

In [8]:
thread = threading.Thread(target=square_threading)
thread.start()

In [9]:
for i in range(10):
    in_queue.put(i)
    i_squared = out_queue.get()
    print(f"{i} squared is {i_squared}")
in_queue.put('STOP')
thread.join()

0 squared is 0
1 squared is 1
2 squared is 4
3 squared is 9
4 squared is 16
5 squared is 25
6 squared is 36
7 squared is 49
8 squared is 64
9 squared is 81


In [10]:
class Primes:
    
    def __init__(self):
        self.current = 2
        
    def __iter__(self):
        return self
    
    def __next__(self):
        while True:
            current = self.current
            square_root = int(current ** 0.5)
            is_prime = True
            if square_root >= 2:
                for i in range(2, square_root + 1):
                    if current % i == 0:
                        is_prime = False
                        break
            self.current += 1
            if is_prime:
                
                return current

In [11]:
import cProfile
import itertools

cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes())]')

         2466 function calls in 0.013 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-10-66fcf727453d>:3(__init__)
        1    0.000    0.000    0.000    0.000 <ipython-input-10-66fcf727453d>:6(__iter__)
     1230    0.013    0.000    0.013    0.000 <ipython-input-10-66fcf727453d>:9(__next__)
     1230    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.000    0.000    0.013    0.013 <string>:1(<listcomp>)
        1    0.000    0.000    0.013    0.013 <string>:1(<module>)
        1    0.000    0.000    0.013    0.013 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [12]:
class Primes2:
    
    def __init__(self):
        self.known_primes=[]
        self.current=2
        
    def __iter__(self):
        return self
    
    def __next__(self):
        while True:
            current = self.current
            prime_factors = [p for p in self.known_primes if current % p == 0]
            self.current += 1
            if len(prime_factors) == 0:
                self.known_primes.append(current)
                
                return current

In [13]:
cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes2())]')

         23708 function calls in 0.357 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1230    0.007    0.000    0.356    0.000 <ipython-input-12-9b6f6d86b803>:10(__next__)
    10006    0.349    0.000    0.349    0.000 <ipython-input-12-9b6f6d86b803>:13(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-12-9b6f6d86b803>:3(__init__)
        1    0.000    0.000    0.000    0.000 <ipython-input-12-9b6f6d86b803>:7(__iter__)
     1230    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.001    0.001    0.357    0.357 <string>:1(<listcomp>)
        1    0.000    0.000    0.357    0.357 <string>:1(<module>)
        1    0.000    0.000    0.357    0.357 {built-in method builtins.exec}
    10006    0.001    0.000    0.001    0.000 {built-in method builtins.len}
     1230    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 

In [14]:
class Primes3:
    
    def __init__(self):
        self.known_primes=[]
        self.current=2
        
    def __iter__(self):
        return self
    
    def __next__(self):
        while True:
            current = self.current
            sqrt_current = int(current**0.5)
            potential_factors = itertools.takewhile(lambda x: x < sqrt_current, self.known_primes)
            prime_factors = [p for p in potential_factors if current % p == 0]
            self.current += 1
            if len(prime_factors) == 0:
                self.known_primes.append(current)
                
                return current

In [15]:
cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes3())]')

         291158 function calls in 0.087 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1265    0.013    0.000    0.086    0.000 <ipython-input-14-5007f6a1a2b2>:10(__next__)
   267345    0.021    0.000    0.021    0.000 <ipython-input-14-5007f6a1a2b2>:14(<lambda>)
    10006    0.052    0.000    0.073    0.000 <ipython-input-14-5007f6a1a2b2>:15(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-14-5007f6a1a2b2>:3(__init__)
        1    0.000    0.000    0.000    0.000 <ipython-input-14-5007f6a1a2b2>:7(__iter__)
     1265    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.001    0.001    0.087    0.087 <string>:1(<listcomp>)
        1    0.000    0.000    0.087    0.087 <string>:1(<module>)
        1    0.000    0.000    0.087    0.087 {built-in method builtins.exec}
    10006    0.001    0.000    0.001    0.000 {built-in method builtins.len}
     1265    0.000    0.000    0.000

In [16]:
class Primes4:
    
    def __init__(self):
        self.known_primes=[]
        self.current=2
        
    def __iter__(self):
        return self
    
    def __next__(self):
        while True:
            current = self.current
            sqrt_current = int(current**0.5)
            potential_factors = itertools.takewhile(lambda x: x < sqrt_current, self.known_primes)
            is_prime = True
            for p in potential_factors:
                if current % p == 0:
                    is_prime = False
                    break
            self.current += 1
            if is_prime == True:
                self.known_primes.append(current)
                return current

In [17]:
cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes4())]')

         64802 function calls in 0.026 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1265    0.020    0.000    0.025    0.000 <ipython-input-16-879f4d8a4bb6>:10(__next__)
    61001    0.005    0.000    0.005    0.000 <ipython-input-16-879f4d8a4bb6>:14(<lambda>)
        1    0.000    0.000    0.000    0.000 <ipython-input-16-879f4d8a4bb6>:3(__init__)
        1    0.000    0.000    0.000    0.000 <ipython-input-16-879f4d8a4bb6>:7(__iter__)
     1265    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.001    0.001    0.026    0.026 <string>:1(<listcomp>)
        1    0.000    0.000    0.026    0.026 <string>:1(<module>)
        1    0.000    0.000    0.026    0.026 {built-in method builtins.exec}
     1265    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Activity 23

In [18]:
import threading
import queue
import cProfile
import itertools
import numpy as np

In [19]:
np.random.seed(0)

In [20]:
in_queue = queue.Queue()
out_queue = queue.Queue()


def random_number_threading():
    while True:
        n = in_queue.get()
        if n == 'STOP':
            return
        random_numbers = np.random.rand(n)
        out_queue.put(random_numbers)

In [21]:
def generate_random_numbers(show_output, up_to):
    thread = threading.Thread(target=random_number_threading)
    thread.start()
    
    for i in range(up_to):
        in_queue.put(i)
        random_nums = out_queue.get()
        
        if show_output:
            print(random_nums)
            
    in_queue.put('STOP')
    thread.join()

In [22]:
generate_random_numbers(True, 10)

[]
[0.5488135]
[0.71518937 0.60276338]
[0.54488318 0.4236548  0.64589411]
[0.43758721 0.891773   0.96366276 0.38344152]
[0.79172504 0.52889492 0.56804456 0.92559664 0.07103606]
[0.0871293  0.0202184  0.83261985 0.77815675 0.87001215 0.97861834]
[0.79915856 0.46147936 0.78052918 0.11827443 0.63992102 0.14335329
 0.94466892]
[0.52184832 0.41466194 0.26455561 0.77423369 0.45615033 0.56843395
 0.0187898  0.6176355 ]
[0.61209572 0.616934   0.94374808 0.6818203  0.3595079  0.43703195
 0.6976312  0.06022547 0.66676672]


In [23]:
cProfile.run('generate_random_numbers(False, 2000)')

         74058 function calls in 0.106 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003    0.106    0.106 <ipython-input-21-b58bc638e4f8>:1(generate_random_numbers)
        1    0.000    0.000    0.106    0.106 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:38(_remove)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:81(add)
     2001    0.004    0.000    0.014    0.000 queue.py:121(put)
     2000    0.007    0.000    0.089    0.000 queue.py:153(get)
     4000    0.001    0.000    0.002    0.000 queue.py:208(_qsize)
     2001    0.001    0.000    0.001    0.000 queue.py:212(_put)
     2000    0.001    0.000    0.001    0.000 queue.py:216(_get)
        1    0.000    0.000    0.000    0.000 threading.py:1012(join)
        1    0.000    0.000    0.000    0.000 threading.py:1050(_wait_for_tstate_lock)
        2    0.000    0.000    0.000    0.000 threadin